# EUNIS Habitat Modeling

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import sys
import os
import joblib
import glob
import shutil

In [ ]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [ ]:
import random

In [ ]:
import rasterio as rio

In [ ]:
graine = 1234

## Data preparation

In [ ]:
targets = ['MA2','N','P','Q','R','S','T','U','V']

### Habitats

CLEANING RULES:
- Specific to each habitat, done prior to the modelling
- Involves the following criteria, to adapt according to the habitat of interest:
    - Min date of observation
    - Location uncertainty: set according to habitat minimum extent
    - Landcover match in the surroundings

In [ ]:
eva_data = pd.read_csv('dataset/EVA_EUNIS.csv',low_memory=False,index_col=0)[['PlotObservationID','source','year','LocationUncertainty','Longitude','Latitude','X','Y','EUNIS','EUNIS1','EUNIS2','EUNIS3']]

In [ ]:
au_data = pd.read_csv('dataset/AUSTRIA_EUNIS.csv',index_col=0)

In [ ]:
au_data_long = au_data.melt(id_vars=['grid','X','Y','CLC'],var_name='EUNIS',value_name='occ').query('occ>0')

In [ ]:
au_data_long['EUNIS']=au_data_long['EUNIS'].replace(['MAa','MA'],'MA2')
au_data_long['EUNIS1']=au_data_long['EUNIS'].apply(lambda x: 'MA2' if 'MA' in x else x[0:1] if x[0:1] in targets else '')
au_data_long['EUNIS2']=au_data_long['EUNIS'].apply(lambda x: '' if len(x)<2 else x[0:2] if 'MA' not in x else '' if len(x)<=3 else x[0:4])
au_data_long['EUNIS3']=au_data_long['EUNIS'].apply(lambda x: '' if len(x)<3 else x[0:3] if 'MA' not in x else '' if len(x)<=4 else x[0:5])

In [ ]:
pt_data = pd.read_csv('dataset/PORTUGAL_EUNIS.csv',index_col=0)

In [ ]:
ifn_data = pd.read_csv('dataset/IFN_EUNIS.csv',index_col=0)

In [ ]:
nlpt_data = pd.read_csv('dataset/NLPT_EUNIS.csv',index_col=0)
nlpt_data['EUNIS']=nlpt_data['EUNIS'].replace(['MAa','MA'],'MA2')
nlpt_data['EUNIS1']=nlpt_data['EUNIS'].apply(lambda x: 'MA2' if 'MA' in x else x[0:1] if x[0:1] in targets else '')
nlpt_data['EUNIS2']=nlpt_data['EUNIS'].apply(lambda x: '' if len(x)<2 else x[0:2] if 'MA' not in x else '' if len(x)<=3 else x[0:4])
nlpt_data['EUNIS3']=nlpt_data['EUNIS'].apply(lambda x: '' if len(x)<3 else x[0:3] if 'MA' not in x else '' if len(x)<=4 else x[0:5])

### Environment

#### Training

In [ ]:
eva_env = pd.read_csv('dataset/EVA_env_full.csv',index_col=0).dropna(subset=['PlotObservationID','Longitude','Latitude'],axis=0)
eva_env['PlotObservationID']=eva_env['PlotObservationID'].astype(int)

In [ ]:
calib_env = eva_env.groupby(['PlotObservationID','Longitude','Latitude','X','Y']).first().reset_index()

In [ ]:
calib_dataset = pd.merge(eva_data,calib_env,on=['PlotObservationID','Longitude','Latitude','X','Y']).drop_duplicates()
calib_dataset = calib_dataset.reset_index(drop=True)

#### Evaluation

In [ ]:
nlpt_env = pd.read_csv('dataset/NLPT_env_full.csv',index_col=0)
nlpt_dataset = pd.merge(nlpt_env, nlpt_data)
nlpt_dataset.columns = ['grid']+nlpt_dataset.columns.tolist()[1:]

In [ ]:
ifn_env = pd.read_csv('dataset/IFN_env_full.csv',index_col=0)
ifn_dataset = pd.merge(ifn_env, ifn_data)
ifn_dataset.columns = ['grid']+ifn_dataset.columns.tolist()[1:]

In [ ]:
pt_env = pd.read_csv('dataset/PORTUGAL_env_full.csv',index_col=0)
pt_dataset = pd.concat([pt_env.loc[pt_data['grid'],:],pt_data.set_index('grid').drop(['CLC','X','Y'],axis=1)],axis=1)
pt_dataset['grid']=pt_dataset.index
pt_dataset.columns=pt_dataset.columns.tolist()[:-5]+['EUNIS','EUNIS1','EUNIS2','EUNIS3','grid']

In [ ]:
au_env = pd.read_csv('dataset/AUSTRIA_env_full.csv',index_col=0)
au_dataset = pd.concat([au_env.loc[au_data_long['grid'].tolist(),:],au_data_long.set_index('grid').drop(['CLC','X','Y'],axis=1)],axis=1)
au_dataset['grid']=au_dataset.index

## Cross-validation settings

In [ ]:
from source.utilities import *

In [ ]:
n_splits = 5

In [ ]:
calib_data['eea_100km'] = calib_data[['X','Y']].apply(lambda arr: set_eea_geom(arr[0],arr[1]),axis=1)
calib_data['grid']=calib_data.index.tolist()

In [ ]:
do_split = False
if do_split:
    for cl in targets:
        print('Generating cross-validation splits for %s'%cl)
        cl_data, cl_split = mlsplit(calib_data,cl,n_splits,level=3 if cl!='P' else 2)
        joblib.dump([cl_data, cl_split],'partitions/%s.joblib'%cl)
        plot_partition(cl_data,level=3 if cl!='P' else 2)

In [ ]:
if do_split:
    for cl in targets:
        cl_data, cl_split = joblib.load('partitions/%s.joblib'%cl)
        plot_partition(cl_data,level=3 if cl!='P' else 2)

## Modelling utilities

In [ ]:
import itertools
import scipy.stats as ss

In [ ]:
from source.models.evaluation import *
from source.models.preprocessing import *
from source.models.calibration import *

In [ ]:
from source.models.baselines import *
from source.habitat_model import *
from source.models.neural_habitat_model import *
from source.models.tabnet_habitat_model import *
from source.models.decision_forest_habitat_models import *
from source.models.gam_models import *

### Data config

In [ ]:
feature_metadata = pd.read_csv('config/feature_metadata.csv')

In [ ]:
eunis_vars = ['EUNIS1','EUNIS2','EUNIS3']
geo_vars = ['X','Y','Longitude','Latitude']

In [ ]:
abio_vars = ['bio01', 'bio04', 'gdd5', 'bio12', 'bio15', 'scd','swe', ###climate variables available in the future
            'distace2FW', 'distance2Coast','inundation_seasonality', ### hydrography
            'TPILF','TRI','aspect',### Topography
            'dr','parmado',   ### Geology
            'AWC','BulkDensity', 'CoarseFragments', 'SandE', 'SiltE', 'ClayE',  ### Physical soil structure
            'pH_H2O', 'C', 'N', 'CEC', 'CaCO3']  ### Chemical properties of the soil

rs_vars =  ['canopy_height', 'lai_spring', 'lai_summer', 'canopy_density', ## structure
            'SOSD', 'LSLOPE', 'AMPL', 'RSLOPE', 'LENGTH', 'TPROD',  ### phenology
            #'perma_water', 'perma_wet', 'temp_water', 'temp_wet', ### hydrography
            'Grassland', 'Tree_cover', 'Built_up', 'Bare_sparse_vegetation', 'Cropland', 
            'Permanent_water_bodies', 'Herbaceous_wetland', 'Shrubland', 'Moss_lichen', 'Snow_ice'
           ]         

tpilf_categs = ['foot slopes','high ridges','local ridges','mid-slope drainages','mid-slope ridges',
                'plains','streams','upland drainage','upper slopes','valleys']

parmado_categs = ['No information','clastic-sedimentary','sedimentary','calcareous','limestone','dolomite','marl',
  'chalk','evaporites','siliceous','igneous','plutonic','volcanic','pyroclastic','metamorphic','marine_deposits',
  'fluvial_deposits','lake_deposits','residual_loam_deposits','residual_clay_deposits','slope_deposits','glacial_deposits',
  'eolian_deposits','organic_material','peat','slime_ooze','anthropo_deposits']

categories = [tpilf_categs,parmado_categs]
cat_vars = ['TPILF','parmado']

pred_vars = abio_vars + rs_vars

### Utilities

In [ ]:
import json

def load_config(algo_name,config_file):
    with open(config_file) as fp:
        param_dict = json.load(fp)
        param_dict.update({
            'inputs':{
                'metadata': feature_metadata,
                'categories': categories,
                'std': True if algo_name in ["gam","mlp","tabnet"] else False
                'onehot': 0 if algo_name in ["xgb","catboost"] else 1 if algo_name in ["rf","gam","mlp"] else 2 if algo_name in ["lgbm","tabnet"] else None
                'groups': None
            }
        })

    return param_dict

#### Train the ensemble habitat model

In [ ]:
def train_classifier_ensemble(cl,cl_dataset,eva_vars,model_tuples,out_folder):
    cl_att = 'EUNIS3' if cl!='P' else 'EUNIS2'
    target_data = cl_dataset.query('%s==%s'%(cl_att,cl_att)).dropna(subset=eva_vars+[cl_att,'fold','dataset'])
    
    for mod, algo in model_tuples:
        params = load_config(algo,'config/%s_config/%s.json'%(cl,mod))
        out_dir = '%s/%s/%s/'%(out_folder,cl,algo)
        os.makedirs(out_dir,exist_ok=True)
        os.makedirs('%s/confusion/'%out_dir,exist_ok=True)
        os.makedirs('%s/diagnosis/'%out_dir,exist_ok=True)

        block_models = []
        block_perfs = []
        block_aucs = []
        block_conf_mats = []    
        
        if os.path.exists('%s/%s.joblib'%(out_dir,algo)):
            print('Already trained, loading ...')
            pretrained = joblib.load('%s/%s.joblib'%(out_dir,algo))
        else:
            pretrained = None
            
        for f in range(n_splits):
            train_data=target_data.query('fold!=@f').copy()

            train_pool = train_data[cl_att].unique().tolist()
            valid_pool = target_data.query('fold==@f')[cl_att].unique().tolist()

            diff = set(valid_pool).difference(train_pool)
            print('Removing unobserved %s classes in the training set from the validation fold %d: '%(cl,f),diff)

            valid_data=target_data.query('fold==@f & %s in @train_pool'%cl_att)

            X_train = train_data[eva_vars]
            y_train = train_data[cl_att]
            X_test = valid_data[eva_vars]
            y_test = valid_data[cl_att] 
            
            if mod=="biogeo":
                rf_model = BiogeoHabitatModel(model_name=algo, problem=cl, param_dict=params)
                if pretrained is not None:
                    rf_model.model = pretrained[f]
                elif os.path.exists('%s/%s_%d.joblib'%(out_dir,algo,f)):
                    rf_model.model = joblib.load('%s/%s_%d.joblib'%(out_dir,algo,f))                   
                else:
                    rf_model.fit(X_train,y_train)            
            elif mod=="rf":
                rf_model = RandomForestHabitatModel(model_name=algo, problem=cl, param_dict=params)
                if pretrained is not None:
                    rf_model.prefit(X_train,y_train)
                    rf_model.model = pretrained[f]
                elif os.path.exists('%s/%s_%d.joblib'%(out_dir,algo,f)):
                    rf_model.prefit(X_train,y_train)
                    rf_model.model = joblib.load('%s/%s_%d.joblib'%(out_dir,algo,f))                   
                else:
                    rf_model.fit(X_train,y_train)
            elif mod=="xgb":
                rf_model = XGBoostHabitatModel(model_name=algo, problem=cl, param_dict=params)
                if pretrained is not None:
                    rf_model.prefit(X_train,y_train)
                    rf_model.model = pretrained[f]
                elif os.path.exists('%s/%s_%d.joblib'%(out_dir,algo,f)):
                    rf_model.prefit(X_train,y_train)
                    rf_model.model = joblib.load('%s/%s_%d.joblib'%(out_dir,algo,f)) 
                else:                
                    rf_model.fit(X_train,y_train, X_val=X_test, y_val=y_test)
                    fig = rf_model.plot_learning()
                    fig.savefig('%s/diagnosis/learning_curve_%d.png'%(out_dir,f))
                  
            elif mod=='lgbm':
                rf_model = LightGBMHabitatModel(model_name=algo, problem=cl, param_dict=params)
                if pretrained is not None:
                    rf_model.prefit(X_train,y_train)
                    rf_model.model = pretrained[f]
                elif os.path.exists('%s/%s_%d.joblib'%(out_dir,algo,f)):
                    rf_model.prefit(X_train,y_train)
                    rf_model.model = joblib.load('%s/%s_%d.joblib'%(out_dir,algo,f)) 
                else:                
                    rf_model.fit(X_train,y_train, X_val=X_test, y_val=y_test)
                    fig = rf_model.plot_learning()
                    fig.savefig('%s/diagnosis/learning_curve_%d.png'%(out_dir,f)) 
                
            elif mod=='catboost':
                rf_model = CatBoostHabitatModel(model_name=algo, problem=cl, param_dict=params)
                if pretrained is not None:
                    rf_model.prefit(X_train,y_train)
                    rf_model.model = pretrained[f]
                elif os.path.exists('%s/%s_%d.joblib'%(out_dir,algo,f)):
                    rf_model.prefit(X_train,y_train)
                    rf_model.model = joblib.load('%s/%s_%d.joblib'%(out_dir,algo,f))
                else:                
                    rf_model.fit(X_train,y_train, X_val=X_test, y_val=y_test)
                    fig = rf_model.plot_learning()
                    fig.savefig('%s/diagnosis/learning_curve_%d.png'%(out_dir,f))                 

            elif mod=="mlp":
                rf_model = NeuralHabitatModel(model_name=algo, problem=cl, param_dict=params,device=device)
                if pretrained is not None:
                    rf_model.prefit(X_train,y_train)
                    rf_model.neural_module = pretrained[f]
                elif os.path.exists('%s/%s_%d.joblib'%(out_dir,algo,f)):
                    rf_model.prefit(X_train,y_train)
                    rf_model.neural_module = joblib.load('%s/%s_%d.joblib'%(out_dir,algo,f))
                else:                
                    rf_model.fit(X_train,y_train, X_val=X_test, y_val=y_test)

                    fig = rf_model.plot_learning()
                    fig.savefig('%s/diagnosis/learning_curve_%d.png'%(out_dir,f))

                    print('Temperature scaling')
                    y_logit = rf_model.predict_logit(X_test)
                    mlp_temp_sc = TemperatureScaling(predictions=y_logit,labels=y_test,as_logit=True)
                    fig1 = mlp_temp_sc.plot_calibration(title='Pre-calibration')
                    fig1.savefig('%s/diagnosis/precalibration_%d.png'%(out_dir,f))

                    optim = mlp_temp_sc.optimize_temperature()
                    fig2 = mlp_temp_sc.plot_calibration('Post-calibration')
                    fig2.savefig('%s/diagnosis/postcalibration_%d.png'%(out_dir,f))

                    rf_model.temperature = mlp_temp_sc.temperature  
                    
            elif mod=="tabnet":
                rf_model = TabNetHabitatModel(model_name=algo, problem=cl, param_dict=params)
                if pretrained is not None:
                    rf_model.prefit(X_train,y_train)
                    rf_model.model = pretrained[f]
                elif os.path.exists('%s/%s_%d.joblib'%(out_dir,algo,f)):
                    rf_model.prefit(X_train,y_train)
                    rf_model.model = joblib.load('%s/%s_%d.joblib'%(out_dir,algo,f))
                else:                
                    rf_model.fit(X_train,y_train, X_val=X_test, y_val=y_test)

                    fig = rf_model.plot_learning()
                    fig.savefig('%s/diagnosis/learning_curve_%d.png'%(out_dir,f))

                    print('Temperature scaling')
                    y_logit = rf_model.predict_proba(X_test)
                    mlp_temp_sc = TemperatureScaling(predictions=y_logit,labels=y_test,as_logit=False)
                    fig1 = mlp_temp_sc.plot_calibration(title='Pre-calibration')
                    fig1.savefig('%s/diagnosis/precalibration_%d.png'%(out_dir,f))

                    optim = mlp_temp_sc.optimize_temperature()
                    fig2 = mlp_temp_sc.plot_calibration('Post-calibration')
                    fig2.savefig('%s/diagnosis/postcalibration_%d.png'%(out_dir,f))

                    rf_model.temperature = mlp_temp_sc.temperature                      
            
            else:
                raise('Unrecognized algorithm')
            
            if pretrained is None:
                rf_perfs, rf_conf_mat = rf_model.evaluate(X_test,y_test)
                rf_perfs['fold'] = f
                rf_perfs['dataset'] = 'GLOBAL'

                rf_conf_mat.to_csv('%s/confusion/conf_mat_%d.csv'%(out_dir,f))
                fig = plot_confusion_matrix(rf_conf_mat, title='%s, %s, fold: %d'%(algo,cl,f),gs=10)
                fig.savefig('%s/confusion/conf_mat_%d.png'%(out_dir,f))

                Y_hat = rf_model.predict_proba(X_test)
                fig, au_scores=mcroc_eval(Y_hat,y_test,'%s - %s, fold: %d'%(algo,cl,f)) 
                fig.savefig('%s/confusion/roc_%d.png'%(out_dir,f))
                au_scores.update({'fold':f, 'dataset': 'GLOBAL', 'algo': algo})
                
                block_perfs.append(rf_perfs)
                block_aucs.append(au_scores)
                block_conf_mats.append(rf_conf_mat)
            
            joblib.dump(rf_model.model if mod!="mlp" else rf_model.neural_module, '%s/%s_%d.joblib'%(out_dir,algo,f),compress=3)
            block_models.append(rf_model)
            
        if pretrained is None:
            if mod=='mlp':
                joblib.dump([block.neural_module for block in ensemble_model.models],'%s/%s.joblib'%(out_dir,algo),compress=3)
            else:
                joblib.dump([block.model for block in ensemble_model.models],'%s/%s.joblib'%(out_dir,algo),compress=3)
                        
        print('Ensemble model')
        ensemble_model = EnsembleHabitatModel(model_names=['%s%d'%(algo,f) for f in range(n_splits)], 
                                              models=block_models, k_list=[3,5,10], 
                                              ensemble_name='%s_ensemble'%algo, problem=cl)
        
        return ensemble_model

#### Ensemble forecasting

In [ ]:
def ensemble_forecasting(ensemble_model,pred_vars,proj_datasets,out_dir):
    for dname, dataset in proj_datasets:
        print(dname)
        Y_raw, Y_score, Y_committee = ensemble_model.predict_proba(dataset[pred_vars])
        Y_raw.to_parquet('%s/%s_raw.parquet'%(out_dir,dname),compression='gzip')
        Y_score.to_parquet('%s/%s_soft_prediction.parquet'%(out_dir,dname),compression='gzip')
        Y_committee.to_parquet('%s/%s_hard_prediction.parquet'%(out_dir,dname),compression='gzip')
        
        del Y_raw, Y_score, Y_committee

#### Visualize predictions

In [ ]:
from shapely.errors import ShapelyDeprecationWarning
import warnings
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 

In [ ]:
def plot_prediction(geo_df,out_folder,dset,algo,palette=None,xlim=None,ylim=None):

    pred = pd.read_csv('%s/%s_soft_prediction.csv'%(out_folder,dset),index_col=0)
    hardpred = pd.read_csv('%s/%s_hard_prediction.csv'%(out_folder,dset),index_col=0)
        
    pred[['Longitude','Latitude']]=geo_df
    pred['EUNIS']=pred[labels].idxmax(axis=1)
    pred['confidence']=pred[labels].max(axis=1)
    pred['consensus']=hardpred[labels].max(axis=1)
    
    if xlim is None:
        xmin, xmax = pred['Longitude'].min(), pred['Longitude'].max()
        xlim = [xmin, xmax]
    
    if ylim is None:
        ymin, ymax = pred['Latitude'].min(), pred['Latitude'].max()
        ylim = [ymin, ymax]
    
    plot_occurrence(pred, lon_var='Longitude', lat_var='Latitude',att='EUNIS',xlim=xlim,ylim=ylim,title='%s predicted %s habitats'%(algo,cl),msize=2,bgd='black', categorical=True, palette=palette)
    plot_occurrence(pred, lon_var='Longitude', lat_var='Latitude',att='confidence',xlim=xlim,ylim=ylim,title='%s predicted %s habitats - confidence'%(algo,cl),msize=2,bgd='black', categorical=False, cmap='viridis')
    plot_occurrence(pred, lon_var='Longitude', lat_var='Latitude',att='confidence',xlim=xlim,ylim=ylim,title='%s predicted %s habitats - consensus'%(algo,cl),msize=2,bgd='black', categorical=False, cmap='viridis')    

## Model config

In [ ]:
model_configurations = 
{
    'MA2':[('rf','wrf'),('xgb','wxgb'),('mlp','mlp3_wldam')],
    'N':[('rf','wrf'),('xgb','wxgb'),('mlp','mlp3_wldam')],
    'P':[('rf','wrf'),('xgb','wxgb'),('mlp','mlp3_wldam')],
    'Q':[('rf','wrf'),('xgb','wxgb'),('mlp','mlpw_wldam') ,('mlp','mlp3_wldam')],
    'R':[('rf','rf'),('xgb','xgb'),('mlp','mlpw_ldam')],
    'S':[('rf','wrf'),('xgb','xgb'),('lgbm','lgbm'),('mlp','mlpw_ldam')],
    'T':[('rf','wrf'),('xgb','xgb'),('lgbm','lgbm'),('mlp','mlp3_fl5')],
    'U':[('rf','rf'),('xgb','xgb'),('lgbm','lgbm'),('mlp','mlp3_ldam')]
}

## Modeling pipeline

<p align="center">
 <img src="training.png" alt="Ensemble model training" width="600" height="600">
</p>

In [ ]:
os.makedirs('habitat_models/full_final/',exist_ok=True)

In [ ]:
cl = 'MA2'  ## Change this to run another habitat formation model

### Setup

In [ ]:
model_tuples = model_configs.get(cl)

In [ ]:
out_folder = 'habitat_models/'%cl
cl_att = 'EUNIS3' if cl!="P" else "EUNIS2"
eval_datasets = [('EVA':calib_dataset[pred_vars]), ##reprojecting on training dataset
                 ('NLPT':nlpt_dataset[pred_vars]), ###projecting on evaluation datasets
                 ('AU',au_dataset[pred_vars]), ###projecting on evaluation datasets
                 ('IFN',ifn_dataset[pred_vars]), ###projecting on evaluation datasets
                 ('PT',pt_dataset[pred_vars]), ###projecting on evaluation datasets7
                ]

In [ ]:
cl_dataset = calib_dataset.query('EUNIS1==@cl')

### Train

In [ ]:
ensemble_model = train_cl_ensemble(cl,cl_dataset,pred_vars,model_tuples,out_folder)

### Forecast on evaluation datasets

In [ ]:
ensemble_forecasting(ensemble_model,pred_vars,eval_datasets,out_dir)

### Evaluate

In [ ]:
for dname, dataset in eval_datasets:
    geo_df = dataset[['Longitude','Latitude']]
    plot_prediction(geo_df,out_folder,dataset)
    Y_score = pd.read_parquet('%s/%s_soft_prediction.parquet'%(out_folder,dname)) ### ensemble mean
    soft_perfs, soft_conf_mat = eval_classifier(y_score=Y_score,y_true=dataset[cl_att], k_list=[3,5,10],model_name='soft_ensemble', super_class=cl)

    soft_perfs.to_csv('%s/%s_perfs.csv'%(out_folder,dname))
    soft_conf_mat.to_csv('%s/%s_confusion.csv'%(out_folder,dname))

## EUNIS projection at EU scale

<p align="center">
 <img src="prediction.png" alt="Ensemble projection" width="600" height="600">
</p>

In [ ]:
import dask.dataframe as dd
from dask_ml.wrappers import ParallelPostFit

### Global settings

In [ ]:
clim_vars = ['bio01','bio04','gdd5','bio12','bio15','scd','swe']
hydro_vars = ['distace2FW','distance2Coast','inundation_seasonality']
topo_vars = ['TPILF','TRI','aspect']
geol_vars = ['dr','parmado']
soil_phys_vars = ['AWC','BulkDensity','CoarseFragments','SandE','SiltE','ClayE']
soil_chem_vars = ['pH_H2O','C','N','CEC','CaCO3']
veg_vars = ['canopy_height','lai_spring','lai_summer','canopy_density','SOSD','LSLOPE','AMPL','RSLOPE','LENGTH','TPROD']
landscape_vars = ['Grassland','Tree_cover','Built_up','Bare_sparse_vegetation','Cropland','Permanent_water_bodies','Herbaceous_wetland','Shrubland','Moss_lichen','Snow_ice']

In [ ]:
n_splits = 5
partition_size = 1500000

In [ ]:
model_folder = 'habitat_models/full_final/'
proj_folder = 'habitat_projection/'

### Utilities

In [ ]:
def load_partition(start,end):
    eu_clim = pd.read_hdf('data/EU100/climate.h5','/data',start=start,stop=end,columns=clim_vars)
    eu_hydro = pd.read_hdf('data/EU100/hydro.h5','/data',start=start,stop=end,columns=hydro_vars)
    eu_topo = pd.read_hdf('data/EU100/topo.h5','/data',start=start,stop=end,columns=topo_vars)
    eu_geol = pd.read_hdf('data/EU100/geology.h5','/data',start=start,stop=end,columns=geol_vars)
    eu_phys_soil = pd.read_hdf('data/EU100/physical_soil.h5',start=start,stop=end,columns=soil_phys_vars)
    eu_chem_soil = pd.read_hdf('data/EU100/chemical_soil.h5',start=start,stop=end,columns=soil_chem_vars)
    eu_veg = pd.read_hdf('data/EU100/vegetation.h5',start=start,stop=end,columns=veg_vars)
    eu_landscape = pd.read_hdf('data/EU100/woco.h5',start=start,stop=end,columns=landscape_vars)
    
    eu_map = pd.concat([eu_clim,eu_hydro,eu_topo,eu_geol,eu_phys_soil,eu_chem_soil,eu_veg,eu_landscape],axis=1)
    
    return eu_map

In [ ]:
def load_full_ensemble(cl,model_tuples,out_folder,n_splits=5):

    block_models = []
    for algo, model_name in model_tuples:
        params = load_config(algo,'config/%s_config/%s.json'%(cl,model_name))
        out_dir = '%s/%s/%s/'%(out_folder,cl,algo)
        for fold in range(n_splits):
            if algo=="xgb":
                habitat_model = XGBoostHabitatModel(model_name='%s_%d'%(model_name,fold), problem=cl, param_dict=params)
                habitat_model.model = joblib.load('%s/%s_%d.joblib'%(out_dir,model_name,f)) 
            
            if algo=="rf":
                habitat_model = RandomForestHabitatModel(model_name='%s_%d'%(model_name,fold), problem=cl, param_dict=params)
                habitat_model.model = joblib.load('%s/%s_%d.joblib'%(out_dir,model_name,f))        
    
            if algo=="lgbm":
                habitat_model = LightGBMHabitatModel(model_name='%s_%d'%(model_name,fold), problem=cl, param_dict=params)
                habitat_model.model = joblib.load('%s/%s_%d.joblib'%(out_dir,model_name,f))
    
            if algo=="mlp":
                habitat_model = NeuralHabitatModel(model_name='%s_%d'%(model_name,fold), problem=cl, param_dict=params)
                habitat_model.model = joblib.load('%s/%s_%d.joblib'%(out_dir,model_name,f))

            if algo=="tabnet":
                habitat_model = TabNetHabitatModel(model_name='%s_%d'%(model_name,fold), problem=cl, param_dict=params)
                habitat_model.model = joblib.load('%s/%s_%d.joblib'%(out_dir,model_name,f))            
                
            block_models.append(habitat_model)
            names_models.append('%s_%d'%(model_name,fold))

    ensemble_model = EnsembleHabitatModel(model_names=names_models, models=block_models, k_list=[3,5,10], ensemble_name='%s_ensemble'%algo, problem=cl)

    return ensemble_model

In [ ]:
def project_partition(partition,cl,model_tuples,model_folder,proj_folder):
    start = partition*size
    end = start+size
    eu_map = load_partition(start,end)

    ensemble_model = generate_ensemble(cl,model_tuples,out_folder,n_splits=5)
    _, Y_score, Y_committee = ensemble_model.predict_proba(eu_map)

    Y_score.to_parquet('%s/%s_soft_prediction_%d.parquet'%(proj_folder,cl,partition))  ### soft ensemble averaging (mean)
    Y_committee.to_parquet('%s/%s_hard_prediction_%d.parquet'%(proj_folder,cl,partition)) ### hard ensemble averaging (committee vote)

### Run projections

We run parallel predictions across partitions and habitat formations on a computing grid. Here's an example to run for one particular run: partition, habitat formation (cl)

In [ ]:
partition = 0
cl = 'R'

In [ ]:
model_tuples = model_configs.get(cl)
project_partition(partition,cl,model_tuples,model_folder,f"{proj_folder}/{cl}")

### Merge partition results

In [ ]:
cl = 'R'

In [ ]:
pattern = f"{proj_folder}/{cl}/soft_prediction_*.parquet"
matching_files = glob.glob(pattern)

full_soft_projection = pd.concat([pd.read_parquet(file) for file in matching_files],axis=0,ignore_index=True)
full_soft_projection.to_parquet(f"{proj_folder}/{cl}_soft_prediction.parquet")

full_hard_projection = pd.concat([pd.read_parquet(file) for file in matching_files],axis=0,ignore_index=True)
full_hard_projection.to_parquet(f"{proj_folder}/{cl}_hard_prediction.parquet")

shutil.rmtree(f"{proj_folder}/{cl}")

## EUNIS wall-to-wall mapping

In [ ]:
from mapping_utilities import *

Ensemble model projections provide for each habitat formation (e.g. Grasslands (R)) across the project extent (EEA39) the probabilities of all habitat classes within the formation such that the probabilities sum to 1. In the following, we use these projections to generate habitat maps.

<p align="center">
 <img src="mapping_workflow.png" alt="Mapping Worflow" width="600" height="600">
</p>

## General preparation

In [ ]:
proj_folder = 'habitat_projection/'
map_folder = 'habitat_maps/'

### Habitat classes

In [ ]:
formations = ['MA2','N','P','Q','R','S','T','U']

with open('config/habitat_classes.json','r') as fp:
    habitat_classes = json.load(fp)

formations = list(habitat_classes.keys())

In [ ]:
map_legend = pd.read_csv('config/habitat_legend.csv')
name2code = map_legend[['Code_name','Code']].set_index('Code_name')['Code'].to_dict()

### Map mask settings

In [ ]:
### These are the valid pixels where projection was done
map_metadata=pd.read_parquet('data/EU100/metadata.parquet') ## coordinates in EPSG:3035
xmin, ymin = map_metadata[['X','Y']].min()
xmax, ymax = map_metadata[['X','Y']].max()

w = (xmax - xmin) // res 
h = (ymax - ymin) // res

print('Indexing')
profile = get_rasterio_profile(count=1,height=h,width=w,bounds=(xmin, ymin, xmax, ymax),epsg=3035,blockxsize=256,blockysize=256,dtype=np.uint8,params={'BIGTIFF':'YES'})
out_fn = 'temporary.tif'
out_raster = rio.open(out_fn, 'w+', **profile)

indices = out_raster.index(x=map_metadata['X'].values,y=map_metadata['Y'].values)
rows = np.minimum(indices[0],int(h-1))
cols = np.minimum(indices[1],int(w-1))

map_metadata['row_idx']=rows
map_metadata['col_idx']=cols

### Generate regional (coastline, endemism to ecoregions) mask layers

Habitat definitions are sometimes by definition associated to a specific ecoregion (inland habitats) or coastlines (coastal habitats). For instance, Baltic coastal meadows are by definition in the Baltic coast, whereas Pannonian sandy steppes are by definition in the Pannonian ecoregion. 

To account for this, we generate spatial masks following coastline layers and ecoregion layers for each habitat class. In the absence of any spatially explicit information in the definition of a given habitat class, we generate an all-ones mask (meaning we're not making any region).

Raster masks (coastline, ecoregions) have been aligned to the mapping mask (corine land cover)

In [ ]:
coastal_areas = rio.open('data/EU100/coastline.tif').read(1)

ecoregions_db = gpd.read_file('data/Ecoregions2017/Ecoregions2017.shp')
ecoregions_db = ecoregions_db.query('ECO_NAME in @ecoreg_list').to_crs(3035)

In [ ]:
map_metadata['coastline'] = coastline[rows,cols]
map_metadata['ecoregion'] = ecoregions[rows,cols]
map_metadata['corine'] = corine[rows,cols]

In [ ]:
with open('crosswalks/coastline_codes.json','r') as fp:
    coastline_nomen = json.load(fp)

In [ ]:
os.makedirs('%s/masks'%map_folder,exist_ok=True)

#### Coastal habitats (MA2, N)

In [ ]:
for cl in ['MA2','N']:
    coastline_mask = pd.read_csv('crosswalks/%s_coastline.csv'%cl,index_col=0)
    labels = coastline_mask.columns.tolist()
    
    all_masks = []
    for cl3 in labels:
        print('Generating mask for %s'%cl3)
        arr_mask = np.zeros_like(coastal_areas)
        sel_codes = [coastline_map.get(x) for x in coastline_mask.query('%s>0'%cl3).index.tolist()]
        for c in sel_codes:
            arr_mask[coastal_areas==c]=1
    
        all_masks.append(arr_mask)
    
    mask_stack = np.stack(all_masks,axis=2)
    mask_stack[coastal_areas==255]=255
    out_arr = np.transpose(mask_stack,[2,0,1])
    
    out_fn = f'{map_folder}/masks/regional_mask_{cl}.tif'

    ###### Prepare raster
    profile = get_rasterio_profile(count=len(labels),height=h,width=w,bounds=(xmin, ymin, xmax, ymax),epsg=3035,blockxsize=256,blockysize=256,
                                   dtype=np.uint8,params={'BIGTIFF':'YES'})
    
    write_geotiff_tags(out_arr, profile, out_fn, nodata=255, tags={'Author': 'Sara Simoussi', 'Organism': 'LECA-CNRS', 'Project':'EO4diversity'})

#### Inland habitats (Q, R, S, T, U)

For inland habitats, except freshwater and man-made vegetation, we use ecoregions to generate regional masks.

In [ ]:
habitat2ecoregion = pd.read_csv('crosswalks/ecoregion_habitat_mask.csv').set_index(['EUNIS1','EUNIS2','EUNIS3'])
ecoreg_list = habitat2ecoregion.columns.tolist()

In [ ]:
os.makedirs('masks/ecoregions_binary',exist_ok=True)

In [ ]:
for cl in formations[2:]:
    cl3_labels = habitat_classes.get(cl)
    l_masks = []
    for cl3 in cl3_labels:
        print(cl3)
        out_file = 'masks/ecoregions_binary/%s.tif'%eunis_cl3
        l_regions = habitat2ecoregion.loc[eunis_cl1].loc[eunis_cl2].loc[[eunis_cl3]].T.query('%s>0'%eunis_cl3).index.tolist()
        reg_df = ecoregions_db.query('ECO_NAME in @l_regions')
        rasterize_shapefile(vect_obj=reg_df,out_file=out_file,mask_file='data/EU100/clc2018.tif')

        arr = rio.open(out_file).read(1)
        l_masks.append(arr)

    ecoreg_stack = np.stack(l_masks,axis=0)
    
    ###### Save mask raster
    out_fn = f'{map_folder}/masks/regional_mask_{cl}.tif'
    profile = get_rasterio_profile(count=len(labels),height=h,width=w,bounds=(xmin, ymin, xmax, ymax),epsg=3035,blockxsize=256,blockysize=256,
                                   dtype=np.uint8,params={'BIGTIFF':'YES'})
    
    write_geotiff_tags(ecoreg_stack, profile, out_fn, nodata=255, tags={'Author': 'Sara Simoussi', 'Organism': 'LECA-CNRS', 'Project':'EO4diversity'})

In [ ]:
shutil.rmtree('masks/ecoregions_binary/')

### Generate land cover masks

In [ ]:
corine_landcover = rio.open('data/EU100/clc2018.tif').read(1)

In [ ]:
corine_mask = pd.read_csv('crosswalks/corine_mask.csv',index_col=0)

In [ ]:
for cl in formations:
    cl3_labels = habitat_classes.get(cl)
    cl3_mask = corine_mask[cl3_labels]
    
    all_masks = []
    for cl3 in labels:
        print('Generating mask for %s'%cl3)
        arr_mask = np.zeros_like(corine_landcover)
        sel_codes = [x for x in corine_mask.query('%s>0'%cl3).index.tolist()]
        for c in sel_codes:
            arr_mask[corine_landcover==c]=1
    
        all_masks.append(arr_mask)
    
    mask_stack = np.stack(all_masks,axis=2)
    mask_stack[corine_landcover==255]=255
    out_arr = np.transpose(mask_stack,[2,0,1])
    
    out_fn = f'{map_folder}/masks/landcover_mask_{cl}.tif'

    ###### Prepare raster
    profile = get_rasterio_profile(count=len(labels),height=h,width=w,bounds=(xmin, ymin, xmax, ymax),epsg=3035,blockxsize=256,blockysize=256,
                                   dtype=np.uint8,params={'BIGTIFF':'YES'})
    
    write_geotiff_tags(out_arr, profile, out_fn, nodata=255, tags={'Author': 'Sara Simoussi', 'Organism': 'LECA-CNRS', 'Project':'EO4diversity'})

## STEP 1: Habitat suitability maps

In [ ]:
os.makedirs('habitat_maps/suitability/',exist_ok=True)

In [ ]:
for cl in formations:
    #### Read ensemble projections
    cl_data = pd.read_parquet(f"{proj_folder}/{cl}_soft_prediction.parquet")

    #### Output file
    out_fn = 'habitat_maps/suitability/%s.tif'%cl
    
    ###### Prepare raster
    profile = get_rasterio_profile(count=len(cl3_labels),height=h,width=w,bounds=(xmin, ymin, xmax, ymax),epsg=3035,blockxsize=256,blockysize=256,
                                   dtype=np.uint8,params={'BIGTIFF':'YES'})
    
    out_raster = rio.open(out_fn, 'w+', **profile)
    out_arr = out_raster.read()
    out_arr[:]=255

    for i, cl3 in enumerate(cl3_labels):
        print(cl3)
        out_arr[i][row_coast,col_coast] = cl_data.loc[idx_coast,cl3].values//100

    min_vals = cl_data.loc[idx_coast,cl3_labels].min()
    max_vals = cl_data.loc[idx_coast,cl3_labels].max()
    min_vals = min_vals//100
    max_vals = max_vals//100

    suit_scales = list(1/max_vals.values)

    write_geotiff_tags(out_arr,
                       profile,
                       out_fn,
                       nodata=255,
                       tags={'Author': 'Sara Simoussi', 'Organism': 'LECA-CNRS-UGA'},
                       scales=suit_scales,
                       offsets=[0]*len(cl3_labels))

## STEP 2: Habitat probability maps

In [ ]:
os.makedirs('habitat_maps/probability/',exist_ok=True)

In [ ]:
for cl in formations:
    cl3_labels = habitat_classes.get(cl)
    out_fn = 'habitat_maps/probability/%s.tif'%cl
    
    suit_arr = rio.open('habitat_maps/suitability/%s_suitability.tif'%cl).read()
    mask_arr = rio.open('masks/regional_mask_%s.tif'%cl).read()

    oom_mask = (suit_arr==255)
    suit_arr[oom_mask] = 0

    filt_arr = suit_arr * mask_arr
    max_vals = filt_arr.max(axis=(1,2))
    suit_scales = list(1/np.maximum(max_vals,1))
    filt_arr[oom_mask] = 255

    profile = get_rasterio_profile(count=len(cl3_labels),height=h,width=w,bounds=(xmin, ymin, xmax, ymax),epsg=3035,blockxsize=256,blockysize=256,
                                   dtype=np.uint8,params={'BIGTIFF':'YES'})

    write_geotiff_tags(filt_arr,
                   profile,
                   out_fn,
                   nodata=255,
                   tags={'Author': 'Sara Simoussi', 'Organism': 'LECA-CNRS-UGA'},
                   #bands_tags=[{'band1':cl3}],
                   scales=suit_scales,
                   offsets=[0]*len(cl3_labels))

## STEP 3: Formation habitat maps (with land use filtering)

In [ ]:
os.makedirs('habitat_maps/topK/',exist_ok=True)

In [ ]:
get_map_code = np.vectorize(lambda c: name2code(cl3_labels[c]))

In [ ]:
for cl in formations:
    cl3_labels = habitat_classes.get(cl)
    out_fn = 'habitat_maps/probability/%s.tif'%cl

    ### Load probability maps
    prob_arr = rio.open('habitat_maps/probability/%s_probability.tif'%cl).read()

    ### Mask nodata areas
    oom_mask = (prob_arr==255)
    prob_arr[oom_mask] = np.nan
    
    ### Load landcover masks
    mask_arr = rio.open('masks/landcover_mask_%s.tif'%cl).read()

    ### Apply filter
    filt_arr = prob_arr * mask_arr
    del prob_arr, mask_arr

    ### Rescale probabilities
    scale_factor = np.nansum(filt_arr)
    scaled_arr = filt_arr / scale_factor
    del scaled_arr

    ### Top-1 decision
    top1_class = scaled_arr.argmax(axis=0,skipna=True)
    top1_confidence = scaled_confidence.max(axis=0)    

    ### Top-2 decision
    is_top1 = (scaled_confidence==top1_confidence.values.reshape(-1,1))
    scaled_confidence[is_top1]=np.nan

    top2_class = scaled_arr.argmax(axis=0,skipna=True)
    top2_confidence = scaled_confidence.max(axis=0)  

    ### Top-3 decision
    is_top2 = (scaled_confidence==top2_confidence.values.reshape(-1,1))
    scaled_confidence[is_top2]=np.nan

    top3_class = scaled_arr.argmax(axis=0,skipna=True)
    top3_confidence = scaled_confidence.max(axis=0) 

    ### Combine top-k maps
    topk = np.stack([top1_class,top2_class,top3_class],axis=0)
    topk_confidence = np.stack([top1_confidence,top2_confidence,top3_confidence],axis=0)

    ### Assign map codes
    topk_codes = get_map_code(topk)
    del topk
    
    profile = get_rasterio_profile(count=3,height=h,width=w,bounds=(xmin, ymin, xmax, ymax),epsg=3035,blockxsize=256,blockysize=256,
                                   dtype=np.uint16,params={'BIGTIFF':'YES'})

    out_fn = 'habitat_maps/topK/%s_topk.tif'%cl
    write_geotiff_tags(topk_codes.astype(np.uint16),
                   profile,
                   out_fn,
                   nodata=65535,
                   tags={'Author': 'Sara Simoussi', 'Organism': 'LECA-CNRS-UGA'},
                   scales=[1,1,1],
                   offsets=[0,0,0])

    profile = get_rasterio_profile(count=3,height=h,width=w,bounds=(xmin, ymin, xmax, ymax),epsg=3035,blockxsize=256,blockysize=256,
                                   dtype=np.uint8,params={'BIGTIFF':'YES'})

    out_fn = 'habitat_maps/topK/%s_topk_confidence.tif'%cl
    write_geotiff_tags((topk_confidence*100).round(0).astype(np.uint8),
                   profile,
                   out_fn,
                   nodata=255,
                   tags={'Author': 'Sara Simoussi', 'Organism': 'LECA-CNRS-UGA'},
                   scales=[1,1,1],
                   offsets=[0,0,0])

## STEP 4: Wall-to-wall habitat map

### Land cover priority rules

In [ ]:
corine_crosswalk = pd.read_csv('crosswalks/corine_crosswalk.csv')

In [ ]:
corine_landcover = rio.open('data/EU100/clc2018.tif').read(1)

In [ ]:
selection_map = np.stack([np.empty_like(corine_landcover)]*3,axis=0,dtype=str)

In [ ]:
for c, priority_order in corine_crosswalk[['corine','Priority_order']]:
    parsed_order = priority_order.split(',')
    top1_formation = parsed_order[0]
    top2_formation = parsed_order[1] if len(parsed_order)>1 else ''
    top3_formation = parsed_order[2] if len(parsed_order)>2 else ''

    sel_mask = (corine_landcover==c)
    selection_map[0][sel_mask] = top1_formation
    selection_map[1][sel_mask] = top2_formation
    selection_map[2][sel_mask] = top3_formation

### Combining topK maps

In [ ]:
w2w_map = np.stack([np.zeros_like(corine_landcover)]*3,axis=0,dtype=np.uint16)
w2w_confidence = np.stack([np.zeros_like(corine_landcover)]*3,axis=0,dtype=np.uint8)

In [ ]:
for cl in formations:
    print('Setting pixels of %s'%cl)
    topk_map = rio.open('habitat_maps/topK/%s_topk.tif'%cl).read()
    topk_confidence = rio.open('habitat_maps/topK/%s_topk_confidence.tif'%cl).read()

    for i in range(3):
        sel_mask = (selection_map[i]==cl)
        w2w_map[i][sel_mask] = topk_map[0][sel_mask]

        sel_mask = (selection_map[i]==cl)
        w2w_confidence[i][sel_mask] = topk_confidence[0][sel_mask]

### Manual setting of non-target habitats

In [ ]:
for cl in ['NODATA','URBAN','SEA_OCEAN','WATER_COURSE','LAKES_RESERVOIRS','TRANSITIONAL_WATER']:
    sel_mask = (selection_map[i]==cl)
    w2w_map[0][sel_mask] = name2code.get(cl)
    w2w_map[0][sel_mask] = 1

### Mapping

In [ ]:
out_fn = 'habitat_maps/wall2wall/wall2wall.tif'
profile = get_rasterio_profile(count=3,height=h,width=w,bounds=(xmin, ymin, xmax, ymax),epsg=3035,blockxsize=256,blockysize=256,
                               dtype=np.uint16,params={'BIGTIFF':'YES'})
write_geotiff_tags(w2w_map.astype(np.uint16),
               profile,
               out_fn,
               nodata=65535,
               tags={'Author': 'Sara Simoussi', 'Organism': 'LECA-CNRS-UGA'},
               scales=[1,1,1],
               offsets=[0,0,0])

In [ ]:
out_fn = 'habitat_maps/wall2wall/wall2wall_confidence.tif'
profile = get_rasterio_profile(count=3,height=h,width=w,bounds=(xmin, ymin, xmax, ymax),epsg=3035,blockxsize=256,blockysize=256,
                               dtype=np.uint8,params={'BIGTIFF':'YES'})
write_geotiff_tags(w2w_confidence.astype(np.uint8),
               profile,
               out_fn,
               nodata=255,
               tags={'Author': 'Sara Simoussi', 'Organism': 'LECA-CNRS-UGA'},
               scales=[1,1,1],
               offsets=[0,0,0])